<a href="https://colab.research.google.com/github/mbentivegna/Burned-Area-Mapping/blob/main/data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initial Authentication & Imports

In [ ]:
# Imports
import geemap
import ee

# Constants
ASSET_PATH = f'projects/my-masters-thesis/assets/'

In [ ]:
# Connect to earth engine
ee.Authenticate()
ee.Initialize(project='my-masters-thesis')



In [ ]:
# Display map
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Plot A Generic Burned Area Mapping

In [ ]:
data = ee.ImageCollection('ESA/CCI/FireCCI/5_1').filterDate('2020-01-01', '2020-12-31')
burned_data = data.select("BurnDate")

most_recent_burn = burned_data.max()

In [ ]:
baVis = {
  'min': 1,
  'max': 366,
  'palette': [
    'ff0000', 'fd4100', 'fb8200', 'f9c400', 'f2ff00', 'b6ff05',
    '7aff0a', '3eff0f', '02ff15', '00ff55', '00ff99', '00ffdd',
    '00ddff', '0098ff', '0052ff', '0210ff', '3a0dfb', '7209f6',
    'a905f1', 'e102ed', 'ff00cc', 'ff0089', 'ff0047', 'ff0004'
  ]
}

In [ ]:
m.addLayer(most_recent_burn, baVis)

# Helper Functions

In [ ]:
# Retrieve a feature collection asset given its corresponding file_name
def retrieve_asset(file_name: str) -> ee.FeatureCollection:
  return ee.FeatureCollection(ASSET_PATH + file_name)

In [ ]:
# Retrieve an image asset given its corresponding file_name
def retrieve_image_asset(file_name: str) -> ee.FeatureCollection:
  return ee.Image(ASSET_PATH + file_name)

In [ ]:
# Saves a provided feature collection as an asset
def save_as_asset(file_name: str, feature_collection: ee.FeatureCollection) -> None:
  task = ee.batch.Export.table.toAsset(
      collection=feature_collection,
      assetId=ASSET_PATH + file_name
  )

  task.start()

In [ ]:
# Plotting function to show the validation samples that were / weren't burned in different colors
def plot_samples_based_on_feature(filter_criteria: str, feature_collection: ee.FeatureCollection, title: str, color: str):
  filter_condition = ee.Filter.equals('Reference', filter_criteria),
  filtered_collection = feature_collection.filter(filter_condition)

  m.addLayer(filtered_collection, {'color': color}, title)

# Import Reference Dataset

In [ ]:
# Retrieve validation set from earth engine servers
validation_dataset = retrieve_asset("reference-dataset")

# Display on the map
plot_samples_based_on_feature("YES", validation_dataset, "Burned", "red")
plot_samples_based_on_feature("NO", validation_dataset, "Unburned", "green")

# Burnt Pixel Windowing Algorithm

In [ ]:
# Load necessary feature collections
boundary = ee.FeatureCollection("users/thetreemap/IDNBorneo_IslandBuffer3km")
index = ee.FeatureCollection("users/thetreemap/IDN_GridIndexRBI500K")

# Once the logic is ready we loop over all the indices to get the full area of indonesia
indexStartID = 11
indexEndID = 11

# Filter the grid index to get the relevant grid cells
indexAOI = index.filter(ee.Filter.gte('OBJECTID', indexStartID))\
                .filter(ee.Filter.lte('OBJECTID', indexEndID))

# Compute the intersection between the boundary and the filtered grid cells
AOI = boundary.geometry().intersection(indexAOI.geometry())

# Visualize both areas and the intersections
m.addLayer(boundary, {'color': 'red'}, 'Boundary with 3 km buffer')
m.addLayer(indexAOI, {'color': 'blue'}, 'Filtered Grid Index')
m.addLayer(AOI, {'color': 'green'}, 'Intersection Area of Interest (AoI)')

In [ ]:
# Additional constants
observationYear = 2019
dt = 2
cloudCover = 80
stepMonth = 3
shadowThresholdB11 = 700

bands_nd = ['B7', 'B11']
bands_nbr = ['B8', 'B12']
bands_rnbr = ['B12', 'B8']

def S2TOC_cloudMask(im):
    # Cloud probability mask
    cloudProbMask = im.select('MSK_CLDPRB').lt(60)

    # Sentinel-2 Scene Classification Layer (SCL) mask
    SCL = im.select('SCL')
    SCL_mask = (SCL.eq(3).Or(SCL.eq(1)).Or(SCL.eq(8)).Or(SCL.eq(9)).Or(SCL.eq(10)).Or(SCL.eq(11))).Not()

    # Shadow mask
    shadow_mask = im.select(['B11']).lt(shadowThresholdB11).Or(im.select(['B4']).lt(200))

    # Normalized Difference indices
    nd = im.normalizedDifference(bands_nd).rename('ND')
    rnbr = im.normalizedDifference(['B12', 'B8']).rename('RNBR')
    nbr = im.normalizedDifference(['B8', 'B12']).rename('NBR')

    # Combined mask
    mask = cloudProbMask.And(shadow_mask.Not()).And(SCL_mask)

    # Water mask
    water = im.select('SCL').remap([1,2,3,4,5,6,7,8,9,10,11], [0,0,0,0,0,1,0,0,0,0,0])
    water = water.eq(1).rename('Water')

    # Morphological operation
    kernel = ee.Kernel.circle(radius=20, units='meters')
    mask = mask.focal_min(kernel=kernel, iterations=1)

    return im.addBands(nd).addBands(rnbr).addBands(nbr).addBands(water).updateMask(mask).copyProperties(im)



In [ ]:
start_date = ee.Date.fromYMD(observationYear, 3, 1)
end_date = ee.Date.fromYMD(observationYear + 1, 2, 1)

# Define the interval in milliseconds (1 day = 86400000 milliseconds)
interval_millis = 86400000 * dt

# Create the list of dates
listDates = ee.List.sequence(start_date.millis(), end_date.millis(), interval_millis)

In [ ]:
def process_date(dd):
    targetDay = ee.Date(dd)

    # Create Composite Before dd
    startDate = targetDay.advance(-stepMonth, 'month')
    endDate = targetDay.advance(-1, 'day')

    S2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterDate(startDate, endDate) \
            .filterBounds(AOI) \
            .filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'less_than', cloudCover) \
            .map(S2TOC_cloudMask)

    S2_before = S2.mean()
    minNBR_1 = S2_before.normalizedDifference(bands_nbr)

    # imageVisParamS2_before = {"bands": ["B4", "B3", "B2"], 'min': 0, 'max': 3000}
    # m.addLayer(S2_before, imageVisParamS2_before, 'Testing_Before')

    # Create Composite After dd
    startDate = targetDay
    endDate = targetDay.advance(stepMonth, 'month')
    S2 = ee.ImageCollection('COPERNICUS/S2_SR') \
            .filterDate(startDate, endDate) \
            .filterBounds(AOI) \
            .filterMetadata('CLOUD_COVERAGE_ASSESSMENT', 'less_than', cloudCover) \
            .map(S2TOC_cloudMask)

    S2_after = S2.filterDate(targetDay, targetDay.advance(1, 'month')).mean()
    # bare_S2_after = S2.map(S2TOC_nbrMask).qualityMosaic('RNBR')
    # veg_S2_after = S2.map(S2TOC_nbrMaskVeg).median()
    # combined_S2_after = ee.ImageCollection([veg_S2_after.toInt32(), bare_S2_after.toInt32()]).mosaic()
    # imageVisParamS2_after = {"bands": ["B4", "B3", "B2"], 'min': 0, 'max': 3000}
    # m.addLayer(S2_after, imageVisParamS2_after, 'Testing_After')

    minNBR_2 = S2_after.normalizedDifference(bands_nbr)

    # Make difference before and after
    diff_minNBR = minNBR_2.subtract(minNBR_1).multiply(-1).rename('diff_nbr')
    doy = ee.Image(targetDay.getRelative('day', 'year')).rename('doy')
    out = S2_after.addBands(S2_before).addBands(diff_minNBR).addBands(doy.int())#.addBands(combined_S2_after)

    return out.set('system:time_start', targetDay.millis())

# Apply the process_date function to each date in the list
out_diff = ee.ImageCollection(listDates.map(process_date))


In [ ]:
out_diff = out_diff.filterDate(start_date, end_date)

# Define visualization parameters
pal = ["00810a", "1bff00", "fff700", "ff0000"]
visMax = {"min": 0, "max": 0.5, "palette": pal}


# Add the maximum difference MinNBR layer to the map
# m.addLayer(out_diff.select('diff_nbr').max().clip(AOI), visMax, 'Maximum difference MinNBR', False)

# Make quality mosaics with the difference image
S2comp_diff = out_diff.qualityMosaic('diff_nbr').clip(AOI)
waterMask = out_diff.select('Water').sum().lte(3)
S2comp_diff = S2comp_diff.updateMask(waterMask)

# Define visualization parameters for before and after disturbance
imageVisParamS2_before = {"opacity": 1, "bands": ["B11_1", "B8_1", "B4_1"], "min": 60, "max": 5500, "gamma": 1}
imageVisParamS2_after = {"opacity": 1, "bands": ["B11", "B8", "B4"], "min": 60, "max": 5500, "gamma": 1}

# Add the before and after disturbance layers to the map
# m.addLayer(S2comp_diff, imageVisParamS2_before, 'S2comp before disturbance', False)
# m.addLayer(S2comp_diff, imageVisParamS2_after, 'S2comp after disturbance', True)

In [ ]:
image_pre = S2comp_diff.select(['B4_1','B5_1','B6_1','B7_1','B8_1','B8A_1','B11_1','B12_1','NBR_1']).rename(['B4','B5','B6','B7','B8','B8A','B11','B12','NBR'])
image_post = S2comp_diff.select(['B4','B5','B6','B7','B8','B8A','B11','B12','NBR'])

export_params_post = {
    'image': image_post,
    'description': 'IDNSentinel2Post',
    'assetId': ASSET_PATH + 'IDN_S2PostBurnImage',
    'scale': 20,
    'region': AOI,
    'maxPixels': 30000000000
}

export_params_pre = {
    'image': image_pre,
    'description': 'IDNSentinel2Pre',
    'assetId': ASSET_PATH + 'IDN_S2PreBurnImage',
    'scale': 20,
    'region': AOI,
    'maxPixels': 30000000000
}

In [ ]:
# # Export images to Google Earth Engine's asset
# task_post = ee.batch.Export.image.toAsset(**export_params_post)
# task_pre = ee.batch.Export.image.toAsset(**export_params_pre)

# # Start the export tasks
# task_post.start()
# task_pre.start()


NameError: name 'export_params_post' is not defined

In [ ]:
image_post = retrieve_image_asset('IDN_S2PostBurnImage')
image_pre = retrieve_image_asset('IDN_S2PreBurnImage')

In [ ]:
imageVisParamS2 = {"opacity": 1, "bands": ["B11", "B8", "B4"], "min": 60, "max": 5500, "gamma": 1}

m.addLayer(image_post, imageVisParamS2, 'S2comp after disturbance')
m.addLayer(image_pre, imageVisParamS2, 'S2comp before disturbance')